<a href="https://colab.research.google.com/github/nnarey/NBA-MVP-Predictor/blob/main/Training_the_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("2008-2023.csv")
df

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'AST')","('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share
0,0,0,Carmelo Anthony,SF,23,DEN,77,2806,21.1,0.568,...,3.4,1.3,0.5,0.492,0.354,0.786,8.2,0.140,2008,NaN
1,1,1,Gilbert Arenas,PG,26,WAS,13,425,18.2,0.529,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2008,NaN
2,2,2,Kobe Bryant,SG,29,LAL,82,3192,24.2,0.576,...,5.4,1.8,0.5,0.459,0.361,0.840,13.8,0.208,2008,NaN
3,3,3,Caron Butler,SF,27,WAS,58,2314,20.7,0.558,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2008,NaN
4,4,4,Vince Carter,SG,31,NJN,76,2959,18.8,0.550,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2008,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,48,48,Fred VanVleet,PG,28,TOR,69,2535,17.0,0.540,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.0
508,49,49,Devin Vassell,SG,22,SAS,38,1178,15.5,0.550,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.0
509,50,50,Franz Wagner,SF,21,ORL,80,2609,15.9,0.589,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.0
510,51,51,Russell Westbrook,PG,34,TOT,73,2126,16.1,0.513,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.0


In [ ]:
df['Voting Share'] = df['Voting Share'].fillna(0)
#just chose these features randomly
features = ["('Per Game', 'PTS')_x", "('Per Game', 'AST')", "('Per Game', 'TRB')", "('Shooting', 'FG%')",	"('Shooting', '3P%')",	"('Shooting', 'FT%')",	"('Advanced', 'WS/48')"]
df[features] = df[features].fillna(0)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV

#just made it random features
features = ["('Per Game', 'PTS')_x", "('Per Game', 'AST')", "('Per Game', 'TRB')", "('Shooting', 'FG%')",	"('Shooting', '3P%')",	"('Shooting', 'FT%')",	"('Advanced', 'WS/48')"]


X_train = df[features]
y_train = df["Voting Share"]

transformer = make_column_transformer(
    (StandardScaler(), features),
    remainder="passthrough"
)

pipeline = make_pipeline(
    transformer,
    KNeighborsRegressor(n_neighbors=5)
)

grid_cv = GridSearchCV(
    pipeline,
    param_grid={
      "columntransformer__standardscaler__with_mean": [True, False],
      "columntransformer__standardscaler__with_std": [True, False],
      "kneighborsregressor__metric": ["euclidean", "manhattan"],
      "kneighborsregressor__weights": ["uniform", "distance"],
      "kneighborsregressor__n_neighbors": range(3, 10)
    },
    scoring="neg_mean_squared_error",
    cv=4
)

grid_cv.fit(
    X_train, y_train
)

grid_cv.best_params_

{'columntransformer__standardscaler__with_mean': True,
 'columntransformer__standardscaler__with_std': True,
 'kneighborsregressor__metric': 'euclidean',
 'kneighborsregressor__n_neighbors': 5,
 'kneighborsregressor__weights': 'distance'}

In [ ]:
#based on the above results we only have to change params for kneighbors

pipeline = make_pipeline(
    transformer,
    KNeighborsRegressor(n_neighbors=5, weights="distance", metric="euclidean")
)

scores = cross_val_score(
    pipeline,
    X=X_train,
    y=y_train,
    cv=10,
    scoring="neg_mean_squared_error"
)

np.sqrt(-scores.mean())

0.143771601939082

0.143771601939082 (copying this from above so you don't have to run code)
U can see what params were best by what I included for the next code block

we can use the root mean squared error, but we really care about if they predicted it in the correct way, because as long as they get all the people in the right spots, that is technically 100% accuracy. For example:

Actual:
Lebron .8
Curry .6

Prediction:
Lebron .7
Curry .65

RMSE might not be 0, but we still have 100% accuracy

Also, it is hard to show how good our model is using RMSE because we are already just predicting based on the points and not who actually gets the MVP.

Random Forest Regressor, cycling through 1-2 params at a time

In [ ]:
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    transformer,
    RandomForestRegressor(random_state=42)  # Ensure reproducibility
)

grid_cv = GridSearchCV(
    pipeline,
    param_grid={
        #"columntransformer__standardscaler__with_mean": [True, False],
        #"columntransformer__standardscaler__with_std": [True, False],
        "randomforestregressor__n_estimators": [100, 200, 300],  # Number of trees in the forest
        #"randomforestregressor__max_depth": [None, 10, 20, 30],  # Maximum depth of the tree
        #"randomforestregressor__min_samples_split": [2, 5, 10],  # Minimum number of samples required to split an internal node
        #"randomforestregressor__min_samples_leaf": [1, 2, 4],  # Minimum number of samples required to be at a leaf node
        #"randomforestregressor__max_features": ['auto', 'sqrt', 'log2']  # The number of features to consider when looking for the best split
    },
    scoring="neg_mean_squared_error",
    cv=4
)

grid_cv.fit(
    X_train, y_train
)

grid_cv.best_params_

{'randomforestregressor__n_estimators': 300}

In [ ]:
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    transformer,
    RandomForestRegressor(random_state=42, n_estimators=300)  # Ensure reproducibility
)

grid_cv = GridSearchCV(
    pipeline,
    param_grid={
        #"columntransformer__standardscaler__with_mean": [True, False],
        #"columntransformer__standardscaler__with_std": [True, False],
        #"randomforestregressor__n_estimators": [100, 200, 300],  # Number of trees in the forest
        "randomforestregressor__max_depth": [None, 10, 20, 30],  # Maximum depth of the tree
        #"randomforestregressor__min_samples_split": [2, 5, 10],  # Minimum number of samples required to split an internal node
        #"randomforestregressor__min_samples_leaf": [1, 2, 4],  # Minimum number of samples required to be at a leaf node
        #"randomforestregressor__max_features": ['auto', 'sqrt', 'log2']  # The number of features to consider when looking for the best split
    },
    scoring="neg_mean_squared_error",
    cv=4
)

grid_cv.fit(
    X_train, y_train
)

grid_cv.best_params_

{'randomforestregressor__max_depth': None}

In [ ]:
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    transformer,
    RandomForestRegressor(random_state=42, n_estimators=300, max_depth=None)
)

grid_cv = GridSearchCV(
    pipeline,
    param_grid={
        #"columntransformer__standardscaler__with_mean": [True, False],
        #"columntransformer__standardscaler__with_std": [True, False],
        #"randomforestregressor__n_estimators": [100, 200, 300],  # Number of trees in the forest
        #"randomforestregressor__max_depth": [None, 10, 20, 30],  # Maximum depth of the tree
        "randomforestregressor__min_samples_split": [2, 5, 10],  # Minimum number of samples required to split an internal node
        "randomforestregressor__min_samples_leaf": [1, 2, 4],  # Minimum number of samples required to be at a leaf node
        #"randomforestregressor__max_features": ['auto', 'sqrt', 'log2']  # The number of features to consider when looking for the best split
    },
    scoring="neg_mean_squared_error",
    cv=4
)

grid_cv.fit(
    X_train, y_train
)

grid_cv.best_params_

{'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__min_samples_split': 10}

In [ ]:
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    transformer,
    RandomForestRegressor(random_state=42, n_estimators=300, min_samples_leaf=1, min_samples_split=10)  # Ensure reproducibility
)

grid_cv = GridSearchCV(
    pipeline,
    param_grid={
        #"columntransformer__standardscaler__with_mean": [True, False],
        #"columntransformer__standardscaler__with_std": [True, False],
        #"randomforestregressor__n_estimators": [100, 200, 300],  # Number of trees in the forest
        #"randomforestregressor__max_depth": [None, 10, 20, 30],  # Maximum depth of the tree
        #"randomforestregressor__min_samples_split": [2, 5, 10],  # Minimum number of samples required to split an internal node
        #"randomforestregressor__min_samples_leaf": [1, 2, 4],  # Minimum number of samples required to be at a leaf node
        "randomforestregressor__max_features": ['auto', 'sqrt', 'log2']  # The number of features to consider when looking for the best split
    },
    scoring="neg_mean_squared_error",
    cv=4
)

grid_cv.fit(
    X_train, y_train
)

grid_cv.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

{'randomforestregressor__max_features': 'auto'}

In [ ]:
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    transformer,
    RandomForestRegressor(random_state=42, n_estimators=300, min_samples_leaf=1, min_samples_split=10)
)

grid_cv = GridSearchCV(
    pipeline,
    param_grid={
        "columntransformer__standardscaler__with_mean": [True, False],
        "columntransformer__standardscaler__with_std": [True, False],
        },
    scoring="neg_mean_squared_error",
    cv=4
)

grid_cv.fit(
    X_train, y_train
)

grid_cv.best_params_

{'columntransformer__standardscaler__with_mean': False,
 'columntransformer__standardscaler__with_std': False}

I don't understand how standard scaler is better without mean and std, that is basically saying to not standard scale this data

In [ ]:
pipeline = make_pipeline(
    transformer,
    RandomForestRegressor(random_state=42, n_estimators=300, min_samples_leaf=1, min_samples_split=10)  # Ensure reproducibility
)
scores = cross_val_score(
    pipeline,
    X=X_train,
    y=y_train,
    cv=10,
    scoring="neg_mean_squared_error"
)

np.sqrt(-scores.mean())

0.13030380634572705

0.13030380634572705


In [ ]:
pipeline = make_pipeline(
    #removing standard
    RandomForestRegressor(random_state=42, n_estimators=300, min_samples_leaf=1, min_samples_split=10)  # Ensure reproducibility
)

scores = cross_val_score(
    pipeline,
    X=X_train,
    y=y_train,
    cv=10,
    scoring="neg_mean_squared_error"
)

np.sqrt(-scores.mean())

0.1302821386973639

0.1302821386973639


Error scores are very close from using standard sacler and not. This could mean putting more weight to things that aren't percentages is better for our given features. Again, these features were just chosen randomly.